In [91]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

from scipy.ndimage import shift
import matplotlib.pyplot as plt
import keras.api._v2.keras as keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D, GlobalMaxPooling2D, Reshape, Lambda
from keras.optimizers.legacy import SGD


We will create a MineSweeper game with a board size of (a,a). The game will be implemented using two matrix of size (a,a), which will be named the Game board and the Mine board. 

The Game board will be a matrix where: -1 represent an covered cell, "@" represent an uncovered cell without any mine in its surrounding, numbers 1-8 represent a cell with the number of bombs surrounding it and "*" represent a mine. It will be the game board observed by the player.

The Mine Board will be a matrix with 0 and 1, where 1 is a bomb and 0 is a safe cell. This matrix will store the mine location on the board.

In [92]:
class minesweeper_game(object):
    def __init__(self, coord = (0,0), len_board = 8, print_board = True, number_mines = 10):
        self.print_board = print_board
        self.len_board = len_board
        self.number_mines = number_mines
        if print_board == True: 
            print("Welcome to MineSweeper!!!")
            print("The following board is your game board. It has {} mines".format(self.number_mines))
            print(np.full((self.len_board,self.len_board), ".", dtype=np.dtype('U100')))
            print("Coordinates ({},{}) revealed:".format(coord[0],coord[1]))
        self.mine_board, self.game_board = self.create_boards(coord)
        self.update_game_board(coord)
        self.uncover_all_empty_spaces()
        if print_board == True:
            print(self.game_board)
        
    def create_boards(self, coord): #We create both boards. We ask for coord so that the first input can not be a loss
        mine_board = np.full((self.len_board,self.len_board),0)  #We define a np.array of size (a,a) with only zeroes
        mines = [] #list of mines
        while len(mines) <= self.number_mines: #we want #self.len_board mines
            mine_coord = (random.randint(0, self.len_board-1), random.randint(0, self.len_board-1)) #we consider the coordinate of the mine
            if (coord != mine_coord) and (mine_coord not in mines): 
                mines.append(mine_coord)
                mine_board[mine_coord] = 1
        game_board = np.full((self.len_board,self.len_board),".") #game board is empty
        self.number_mines = len(mines)
        return mine_board, game_board
                    
    def game_status(self, coord): #Check the state of the game
        if self.mine_board[coord] == 1: #Uncovered mine = loss
            return 0
        uncovered_cells = self.len_board*self.len_board
        for i in range(0,self.len_board): #This for can be simplified using for iy, ix in np.ndindex(a.shape) to loop in all elements
            for j in range(0, self.len_board):
                if (self.game_board[(i,j)] != "."):
                    uncovered_cells -= 1
        if uncovered_cells == self.number_mines:
            return 1 #You uncovered all of the cells without a bomb (There are "a" bombs).
        else:
            return 2 #The game is still going
    
    def update_game_board(self, coord): #After the player gives a coordinates, update the map with the new info
        i = coord[0]
        j = coord[1]
        if self.mine_board[coord] == 1: #Bomb
            self.game_board[coord] = "*"
        else:
            count_mines = 0
            #We have to separate the cases on which (a,b) it is in the border because we have to add the surrounding number of bombs
            if i == 0: #First row 
                for m in range(0, 2): 
                    if j == 0: #Corner (0,0)
                        for n in range(0, 2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    elif j == self.len_board-1: #Corner (0,len)
                        for n in range(j-1, j+1):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    else: #row x = 0 without corners
                        for n in range(j-1, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
            elif i == self.len_board-1: #Last row
                for m in range(i-1, i+1):
                    if j == 0: #corner (len, 0)
                        for n in range(j, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    elif j == self.len_board-1: #corner (len, len)
                        for n in range(j-1, j+1):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    else: #row x = len without corners
                        for n in range(j-1, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
            else: #Not in first or last row
                for m in range(i-1, i+2):
                    if j == 0: #First column
                        for n in range(j, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    elif j == self.len_board-1: #Last column
                        for n in range(j-1, j+1):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
                    else:  #points is in the middle
                        for n in range(j-1, j+2):
                            if (self.mine_board[(m,n)] == 1) and ((m,n) != (i,j)):
                                count_mines += 1
            if count_mines == 0: #No mines = empty
                self.game_board[coord] = "@"
            else:
                self.game_board[coord] = count_mines #Number of mines surrounding it

    def uncover_all_empty_spaces(self): #If the coordinate added was a "@", we clear all "@" and number surrounding it.
        #We want to consider only the cells which are above, below and in the sides of a "@".
        covered_empty = True
        local_game_board = np.copy(self.game_board) #Copy of the original board, to compare
        while covered_empty == True: 
            for i in range(0,self.len_board): #Rows
                for j in range(0,self.len_board): #Columns
                    if self.game_board[(i,j)] == "@":
                        if i == 0:
                            for m in range(0, 2): #we can only take row 0 and 1
                                if j == 0: #Corner (0,0)
                                    for n in range(0, 2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (1,1))): 
                                            self.update_game_board((m,n))
                                elif (j == self.len_board-1): #Corner (0,len)
                                    for n in range(self.len_board-2, self.len_board):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (1,self.len_board-2))):
                                            self.update_game_board((m,n))
                                else: #row x = 0 without corners
                                    for n in range(j-1, j+2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (1,j-1)) and ( (m,n) != (1,j+1))):
                                            self.update_game_board((m,n))
                        elif i == self.len_board-1:
                            for m in range(self.len_board-2, self.len_board):
                                if j == 0: #corner (len, 0)
                                    for n in range(0, 2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (self.len_board-2, 1))):
                                            self.update_game_board((m,n))
                                elif j == self.len_board-1: #corner (len, len)
                                    for n in range(self.len_board-2, self.len_board):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (self.len_board-2, self.len_board-2) )):
                                            self.update_game_board((m,n))
                                else: #row x = len without corners
                                    for n in range(j-1, j+2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1, j-1)) and ( (m,n) != (i-1,j+1))) :
                                            self.update_game_board((m,n))
                        else:
                            for m in range(i-1, i+2):
                                if j == 0: #First column
                                    for n in range(0, 2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1,1) ) and ((m,n) != (i+1,1))):
                                            self.update_game_board((m,n))
                                elif j == self.len_board-1: #Last column
                                    for n in range(self.len_board-2, self.len_board):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1, self.len_board-2) ) and ( (m,n) != (i+1, self.len_board-2))):
                                            self.update_game_board((m,n))
                                else: #coordinate is in the middle
                                    for n in range(j-1, j+2):
                                        if ((self.mine_board[(m,n)] != 1) and ((m,n) != (i-1, j-1) ) and ((m,n) != (i-1, j+1) ) and ((m,n) != (i+1, j-1) ) and ((m,n) != (i+1, j+1) )):
                                            self.update_game_board((m,n))
                    else:
                        continue
                else:
                    continue
            if np.array_equal(local_game_board, self.game_board):
                #To stop we check that the last loop didnt added a new cleared element.
                covered_empty = False
            else:
                local_game_board = np.copy(self.game_board) #If we added new cells, we update the local_game_board to compare again after the loop.
                
                
    def enter_coord(self, coord): #Method to reveal a new cell given a coordinate.
        i = coord[0]
        j = coord[1]
        i = int(i)
        j = int(j)
        coord = (i,j)
        if self.print_board == True:
            print("Coordinates ({},{}) revealed:".format(i,j))
        self.update_game_board(coord)
        self.uncover_all_empty_spaces()
        if self.print_board == True:
            print(self.game_board)
        game_status = self.game_status(coord)
        return game_status, self.game_board

    def check_if_loses(self, game_status): #Method just to print the status of the game.
        if game_status == 0:
            print("You lost!. You found a mine.")
        elif game_status == 1:
            print("You won!. You uncovered all the mines.")
        else:
            print("There are still mines left.")

#Helpful functions:
def input_int(): #To check that the input is the desire type
    Not_integer = True
    while Not_integer == True:
        try:
            len_board = int(input("Enter a number: "))
            return int(len_board)
        except ValueError:
            print("Input is not an integer")
    
def input_coord(): #To check that the input is the desire type
    Not_tuple = True
    while Not_tuple == True:
        try:
            coord = input("Enter a tuple of the form a,b: ").split(",")
            i = coord[0]
            j = coord[1]
            i = int(i)
            j = int(j)
            return (i,j)
        except ValueError:
            print("Input is not a tuple of two integers in the form a,b")

The MineSweeper works in the following way:

First, we create a game with
    game = minesweeper_game(len_board, coord)
The len_board is the length of the arrows (or columns) of the matrix, while the coord is the coordinate of the first revealed cell to generate the map.

Then, we start revealing more coordinates with
    game.enter_coord((a,b))
This method accept a tuple and returns the updated board game, and the game_status to check if it has won, lost or is still going.

In [93]:
game = minesweeper_game(len_board = 8, coord = (5,1))
game_status, game_board = game.enter_coord((0,0))
game.check_if_loses(game_status)

Welcome to MineSweeper!!!
The following board is your game board. It has 10 mines
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (5,1) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '2' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (0,0) revealed:
[['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '2' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['1' '2' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
There are still min

Now, we want to create a program, which will have a Convolutional Neural Networks on its core, to learn how to play minesweeper. For this we will construct the following program:

1. A Possible_moves function, which given a board_game will give a List of every coordinate of each covered cell.
2. An Evaluator, that will be a Neural Network, which given a board_game with the dictionary of coordinates of covered cell, will evaluate each member of the dictionary and asigning a value between 0 and 1.
3. A Coordinate_selector, which will select the coordinate with the highest value and will apply it.

We start with the Possible_moves function.

In [94]:
def possible_moves(game_board):
    len_board = game_board.shape[0]
    possible_moves_list = []
    for i in range(0,len_board): #Rows
        for j in range(0,len_board): #Columns
            if game_board[(i,j)] == ".":
                possible_moves_list.append((i,j))
    return possible_moves_list #List of every non checked cell.

In [95]:
game = minesweeper_game(len_board = 8, coord = (5,1))
game_status, game_board = game.enter_coord((0,0))
possible_moves_list = possible_moves(game_board)
print(possible_moves_list) #It works.

Welcome to MineSweeper!!!
The following board is your game board. It has 10 mines
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (5,1) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '1' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (0,0) revealed:
[['@' '@' '@' '@' '1' '.' '.' '.']
 ['@' '1' '2' '2' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '2' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['1' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
[(0, 5), (0, 6), (0

We follow with the Evaluator:

It will be a Convolutional Neural Network which will asign a value to every coordinate of the board, and we will pick the highest value.

In [ ]:
def CNNmodel(length):
    model = Sequential()
    model.add(Input(shape=(length+4, length+4, 12)))
    model.add(Conv2D(16, kernel_size=(5, 5), padding="valid", activation="relu"))  #Valid to reduce the shape to (length, length)
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=(5, 5), padding="same", activation="relu"))  
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(5, 5), padding="same", activation="relu"))  
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(5, 5), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size=(5, 5), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(1, kernel_size=(3, 3), padding="same", activation="sigmoid"))
    model.compile(loss='mean_squared_error', optimizer="adam", metrics=['accuracy'])
    return model


Then we combine the previous two function on a Coordinate_selector

In [97]:
def extend_board(board, len_board):
    extended_board = np.full((len_board+4,len_board+4), 11)
    for i in range(0,len_board): #Rows
        for j in range(0,len_board): #Columns
            extended_board[i+2,j+2] = board[i,j]
    return extended_board

def board_to_numbers_extended_one_shot(game_board):
    length = game_board.shape[0]
    board_numbers = game_board.copy()
    board_numbers[board_numbers == "."] = 10
    board_numbers[board_numbers == "@"] = 0 
    board_numbers[board_numbers == "*"] = 9
    board_numbers = board_numbers.astype(dtype='int') #Then, when every element is an integer, we transform to dtype int
    num_categories = 12
    one_hot_dict = {i: np.eye(num_categories)[i].tolist() for i in range(num_categories)}
    board_extended = extend_board(board_numbers, length)
    board_hot_encodded = np.array([[one_hot_dict[value] for value in row] for row in board_extended.reshape(length+4, length+4)]) 
    return np.expand_dims(board_hot_encodded, axis=0) #Numerical board to give to the neural network


def coordinate_selector(model, game_board): #Grab the board, compute possible moves and asign to each move a value
    game_board_num = board_to_numbers_extended_one_shot(game_board)
    tensor = tf.convert_to_tensor(game_board_num, dtype=tf.float32)
    possible_moves_list = possible_moves(game_board)
    score = model.predict(tensor, verbose = 0)[0, :, :, 0]
    score_possible_moves = [score[i, j] for i, j in possible_moves_list]
    selected_coord = possible_moves_list[np.argmax(score_possible_moves)]
    return selected_coord, score, game_board_num

We are ready to test our program. It should return a selected_coord which we will input in our enter_coord method, and a score.

In [98]:
game = minesweeper_game(len_board = 8, coord = (2,3))
game_status, game_board = game.enter_coord((0,0))
model = CNNmodel(game.len_board)
selected_coord, score, game_board_num = coordinate_selector(model, game_board)
print("Selected coord: ",selected_coord)
print("Score: ", score[selected_coord])
game_status, game_board = game.enter_coord(selected_coord)
selected_coord, score, game_board_num = coordinate_selector(model, game_board)
print("Selected coord: ",selected_coord)
print("Score: ", score[selected_coord])
game_status, game_board = game.enter_coord(selected_coord)

Welcome to MineSweeper!!!
The following board is your game board. It has 10 mines
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (2,3) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '1' '2' '.' '.' '.' '.']
 ['1' '1' '@' '@' '1' '.' '.' '.']
 ['@' '@' '@' '@' '1' '.' '.' '.']
 ['@' '1' '1' '1' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '@' '1' '.' '.' '.' '.' '.']]
Coordinates (0,0) revealed:
[['1' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '1' '2' '.' '.' '.' '.']
 ['1' '1' '@' '@' '1' '.' '.' '.']
 ['@' '@' '@' '@' '1' '.' '.' '.']
 ['@' '1' '1' '1' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '1' '.' '.' '.' '.' '.' '.']
 ['@' '@' '1' '.' '.' '.' '.' '.']]
Selected coord:  (6

We can verify that the minesweeper board is invariant to the action of the Dihedral group $D_4$, thus for each score we really have 8 possible board to teach my model.

In [99]:
def D4_scores(score):
    scores = [score]
    transpose = tf.transpose(score)
    scores.append(transpose)
    score = tf.expand_dims(score, axis=-1)
    for i in range(3):
        score = tf.image.rot90(score, k = i)
        scores.append(tf.squeeze(score, axis=-1))
    transpose = tf.expand_dims(transpose, axis=-1)
    for i in range(3):
        transpose = tf.image.rot90(transpose, k = i)
        scores.append(tf.squeeze(transpose, axis=-1))
    return scores

def D4_board(board):
    transpose = tf.transpose(board, perm=[0, 2, 1, 3])
    stacked = tf.concat([board, transpose], axis=0)
    for i in range(3):
        board = tf.image.rot90(board, k = i)
        stacked = tf.concat([stacked, board], axis=0)
    for i in range(3):
        transpose = tf.image.rot90(transpose, k = i)
        stacked = tf.concat([stacked, transpose], axis=0)
    return stacked

We are ready to implement the self-learning algorithm.

In [109]:
def train(model, len_board, print_progress = False, print_board = True, number_mines = 10):
    if print_progress==True:
        print("___________________________________________________________________")
        print("Starting a new game")
    first_coord = (random.randint(0, len_board-1), random.randint(0, len_board-1))
    game = minesweeper_game(len_board = len_board, coord = first_coord, print_board = print_board, number_mines = number_mines)
    game_status = 2
    scores_list=[]
    corrected_scores_list = []
    board_states = []
    number_of_turns = 0
    while game_status == 2:
        selected_coord, score, game_board_num = coordinate_selector(model, game.game_board)
        game_status, game.game_board = game.enter_coord(selected_coord)
        final_score = score.copy()
        scores_list += D4_scores(score.copy())
        if game_status == 2:
            score[(selected_coord[0], selected_coord[1])] = (1.05**number_of_turns)
        elif game_status == 1:
            score[(selected_coord[0], selected_coord[1])] = (2**number_of_turns)
        else: 
            score[(selected_coord[0], selected_coord[1])] = -(1.05**number_of_turns)
        corrected_scores_list += D4_scores(score.copy())
        board_states.append(D4_board(game_board_num))
        number_of_turns += 1
    if game_status == 0:
        result = "Loss"
    elif game_status == 1:
        result = "Won"
    if print_progress==True:
        print("Program has ",result)
        print("\n Correcting the Scores and Updating the model weights")
        print("___________________________________________________________________\n")
    x = tf.concat(board_states, axis = 0)
    y = tf.convert_to_tensor(corrected_scores_list)
    model.fit(x, y, epochs=1, batch_size=8, verbose=0)
    return result, final_score
    

We check that the train algorithm work:

In [110]:
result, final_score =train(model, len_board = 8, print_progress=True)
print(final_score)
print(selected_coord)
print(final_score[selected_coord])

___________________________________________________________________
Starting a new game
Welcome to MineSweeper!!!
The following board is your game board. It has 10 mines
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (4,4) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '1' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (3,5) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '*' '.' '.']
 ['.' '.' '.' '.' '1' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 [

Finally, we start the training:

In [111]:
game_counter=0
data_for_graph=pd.DataFrame()
number_of_wins = 0
len_board = 9
model = CNNmodel(len_board)

while(game_counter<=1000):
    if game_counter % 100 == 0:
        print("Game#: ",game_counter, "Game won: ", number_of_wins)
        print_board = True
    else:
        print_board = False
    result=train(model, len_board, print_progress=False, print_board = print_board, number_mines = 10)
    if game_counter % 100 == 0:
        print("final score: ", final_score)
    #data_for_graph=pd.concat([data_for_graph, pd.Series({"game_counter":game_counter,"result":result})])
    #print(game_counter)
    if result == "Won":
        number_of_wins += 1 
        print("The program won.")
    game_counter+=1
    
model.save('models/model_0.keras') 
    

Game#:  0 Game won:  0
Welcome to MineSweeper!!!
The following board is your game board. It has 10 mines
[['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (1,3) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '1' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.']]
Coordinates (2,5) revealed:
[['.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '1' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '1' '.' '.' '.']
 ['.' '.' '.' '.' 